In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [3]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [4]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [5]:
data_fund_org_old = pd.read_excel('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING_old/Funding_Name_Preferred_list_add.xls')#.set_index('Funding Name Preferred')
data_fund_org_old#.head()

,Funding Name Preferred,785,Organization Country,Country,Type of funder,Website
0,nih - usa),74.0,National Institutes of Health (NIH),USA,Public,NaN
1,defense threat reduction agency,45.0,U.S. Department of Defense (US DoD),USA,Public,www.defense.gov/
2,national institutes of health - usa),44.0,National Institutes of Health (NIH),USA,Public,NaN
3,national institute of allergy and infectious d...,35.0,National Institutes of Health (NIH),USA,Public,NaN
4,niaid nih hhs - usanih national institute of a...,27.0,National Institutes of Health (NIH),USA,Public,NaN
...,...,...,...,...,...,...
1321,insermEuropean Commission),NaN,Institut national de la santé et de la recherc...,France,Public,www.inserm.fr/
1322,public health agency of canada - canadian saf...,NaN,Centre for Security Science,Canada,NaN,NaN
1323,united states centres for disease control and ...,NaN,Centers for Disease Control and Prevention (CDC),USA,Public,www.cdc.gov/
1324,united states centres for disease control and ...,NaN,Centers for Disease Control and Prevention (CDC),USA,Public,www.cdc.gov/


In [6]:
data_fund_org = pd.read_csv('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/Funding_Name_Preferred_list.csv').rename(columns={'Unnamed: 0':'Funding Name Preferred'})#.set_index('Funding Name Preferred')
data_fund_org#.head()

,Funding Name Preferred,0
0,Funding Name Preferred,575
1,NaN,21
2,'emmy noether' fellowship from deutsche forsch...,1
3,'interne fondsen ku leuven/internal funds ku l...,1
4,academy of finland (aka)(academy of finlandfin...,1
...,...,...
961,world health organization (who)(world health o...,2
962,world health organization(world health organiz...,4
963,young elite scientist sponsorship program of cast,1
964,youth innovation promotion association cas,1


In [7]:
data_fund_org_new = data_fund_org.merge(data_fund_org_old, 
                                        on='Funding Name Preferred', how='left')
data_fund_org_new

,Funding Name Preferred,0,785,Organization Country,Country,Type of funder,Website
0,Funding Name Preferred,575,NaN,NaN,NaN,NaN,NaN
1,NaN,21,26.0,/,NaN,NaN,NaN
2,'emmy noether' fellowship from deutsche forsch...,1,NaN,NaN,NaN,NaN,NaN
3,'interne fondsen ku leuven/internal funds ku l...,1,1.0,NaN,NaN,NaN,NaN
4,academy of finland (aka)(academy of finlandfin...,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
965,world health organization (who)(world health o...,2,NaN,NaN,NaN,NaN,NaN
966,world health organization(world health organiz...,4,NaN,NaN,NaN,NaN,NaN
967,young elite scientist sponsorship program of cast,1,1.0,China Association for Science and Technology,China,NaN,http://english.cast.org.cn/col/col471/index.ht...
968,youth innovation promotion association cas,1,2.0,Chinese Academy of Sciences,China,NaN,NaN


In [8]:
# save data
data_fund_org_new.to_csv(f'../TABLES/data_fund_org_new_to_clean.csv')

# Import clean data

In [9]:
data_fund_org_clean = pd.read_excel('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_org_new_to_clean_bon.xls')#.set_index('Funding Name Preferred')
data_fund_org_clean['Organisation_clean_and_country'] = data_fund_org_clean['Organisation_clean_and_country'].str.lower()
data_fund_org_clean['Country'] = data_fund_org_clean['Country'].str.lower()
data_fund_org_clean['region'] = data_fund_org_clean['region'].str.lower()
data_fund_org_clean#.head()

,Unnamed: 0,Funding Name Preferred,0,785,Organisation_clean_and_country,Unnamed: 5,Organization Country2,Unnamed: 7,Organization Country,Country,region,Country2,Type of funder,Unnamed: 13,Website
0,42.0,basler,1.0,1.0,/ - /,-,/,NaN,NaN,/,/,/,NaN,NaN,NaN
1,55.0,bill amp,1.0,1.0,/ - /,-,/,NaN,NaN,/,/,/,NaN,NaN,NaN
2,171.0,department of microbiology and immunology,1.0,1.0,/ - /,-,/,NaN,NaN,/,/,/,NaN,NaN,NaN
3,277.0,foundation for the fusion of science and techn...,1.0,1.0,/ - /,-,/,NaN,NaN,/,/,/,NaN,NaN,NaN
4,327.0,group-type special supporting project for educ...,1.0,1.0,group-type special supporting project for educ...,-,group-type special supporting project for educ...,NaN,NaN,/,/,/,University,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,951.0,washington university chancellor's fellowship ...,1.0,1.0,washington university - usa,-,Washington University,NaN,Washington University,usa,north america,USA,University,NaN,NaN
975,959.0,whitehead fellows program,1.0,1.0,whitehead institute for biomedical research - usa,-,Whitehead Institute for Biomedical Research,NaN,Whitehead Institute for Biomedical Research,usa,north america,USA,NaN,NaN,NaN
976,961.0,william and ella owens medical research founda...,1.0,1.0,william and ella owens medical research founda...,-,William and Ella Owens Medical Research Founda...,NaN,William and Ella Owens Medical Research Founda...,usa,north america,USA,NaN,NaN,NaN
977,962.0,william j. fulbright foundation,1.0,1.0,fulbright foundation - usa,-,Fulbright Foundation,NaN,Fulbright Foundation,usa,north america,USA,NaN,NaN,NaN


In [10]:
data_fund_org_clean['Organisation_clean_and_country'].value_counts()

national institute of allergy and infectious diseases - usa                        76
defense threat reduction agency - usa                                              39
national institutes of health - usa                                                20
centers for disease control and prevention - usa                                   20
german research foundation - germany                                               17
                                                                                   ..
science and technology research partnership for sustainable development - japan     1
santen inc. - japan                                                                 1
saint luke university - japan                                                       1
research program on emerging and re-emerging infectious diseases - japan            1
economic community of west african states - west africa                             1
Name: Organisation_clean_and_country, Length: 458, dty

In [11]:
data_fund_org_clean['Organisation_clean_and_country'].value_counts().head(50)

national institute of allergy and infectious diseases - usa                                       76
defense threat reduction agency - usa                                                             39
national institutes of health - usa                                                               20
centers for disease control and prevention - usa                                                  20
german research foundation - germany                                                              17
japan agency for medical research and development - japan                                         17
ministry of education, culture, sports, science and technology - japan                            17
european commission - european union                                                              12
united states department of energy - usa                                                          11
united states agency for international development - usa                                   

In [12]:
data_fund_org_dummies = pd.read_csv('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/Funding_Name_Preferred.csv').set_index(['UT (Unique WOS ID)','Funding Name Preferred']).drop('Unnamed: 2', axis=1)
data_fund_org_dummies#.head()

,,'emmy noether' fellowship from deutsche forschungsgemeinschaft(german research foundation (dfg)),'interne fondsen ku leuven/internal funds ku leuven' project,academy of finland (aka)(academy of finlandfinnish funding agency for technology & innovation (tekes)),administration for strategic preparedness and response,"aethlon medical inc., san diego, usa",african academy of sciences (aas)'s alliance for accelerating excellence in science in africa (aesa),african field epidemiology network,african field epidemiology network (afenet),agence nationale de la recherche(french national research agency (anr)),agency for medical research and development,...,who(world health organization),william and ella owens medical research foundation,william j. fulbright foundation,world bank project,world health organization (who) collaborating center for arboviruses and hemorrhagic fever viruses at the bernhard nocht institute for tropical medicine,world health organization (who)(world health organization),world health organization(world health organization),young elite scientist sponsorship program of cast,youth innovation promotion association cas,zhejiang university special scientific research fund for covid-19 prevention and control
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000849602200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
 # rename organisation names on columns
old_names = data_fund_org_clean['Funding Name Preferred'].to_list()
new_names  = data_fund_org_clean['Organisation_clean_and_country'].to_list()
data_fund_org_dummies_col_replace = data_fund_org_dummies.rename(columns=dict(zip(old_names, new_names)))#.astype(int)    
data_fund_org_dummies_col_replace

,,german research foundation - germany,ku leuven internal funds - belgium,academy of finland - finland,administration for strategic preparedness and response - usa,"aethlon medical, inc. - usa",african academy of sciences - african union,african field epidemiology network - nigeria,african field epidemiology network - nigeria,agence nationale de la recherche - france,japan agency for medical research and development - japan,...,world health organization - multilateral,william and ella owens medical research foundation - usa,fulbright foundation - usa,world bank - multilateral,world health organization - multilateral,world health organization - multilateral,world health organization - multilateral,china association for science and technology - china,chinese academy of sciences - china,zhejiang university - china
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000849602200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data_fund_org_dummies.columns.value_counts()

'emmy noether' fellowship from deutsche forschungsgemeinschaft(german research foundation (dfg))    1
national natural science foundation of china international cooperation and exchange program         1
new england primate center                                                                          1
new england regional center for excellence/biodefense and emerging infectious disease               1
new england regional center of excellence-kasper                                                    1
                                                                                                   ..
hannover biomedical research school                                                                 1
hauptman-woodward medical research institute                                                        1
hbrs/center for infection biology/dewin                                                             1
health and labor sciences research grants, japan                                  

In [15]:
data_fund_org_dummies_col_replace.columns.value_counts()

national institute of allergy and infectious diseases - usa                76
defense threat reduction agency - usa                                      39
centers for disease control and prevention - usa                           20
national institutes of health - usa                                        20
ministry of education, culture, sports, science and technology - japan     17
                                                                           ..
center for research on influenza pathogenesis - usa                         1
joint product management office of biodefense therapeutics (bdtx) - usa     1
joint ministerial program of r&d against cbrne risks - usa                  1
johnson & johnson - usa                                                     1
zhejiang university - china                                                 1
Length: 455, dtype: int64

In [16]:
data_fund_org_dummies_col_replace.columns.value_counts().to_frame().to_csv(f'../TABLES/data_fund_org_dummies_col_replace_verif.csv')

In [17]:
# merge and sum the columns with the same countries name
data_fund_org_dummies_col_replace_grp = pd.DataFrame()
df_org_clean2 = pd.DataFrame()
for elt in  data_fund_org_dummies_col_replace.columns.unique():
    col_array = data_fund_org_dummies_col_replace.columns.str.fullmatch(elt) # contains
    data_fund_org_dummies_col_replace_grp[elt] = data_fund_org_dummies_col_replace.loc[:, col_array].sum(axis=1).to_frame();
data_fund_org_dummies_col_replace_grp = pd.concat([data_fund_org_dummies_col_replace_grp, df_org_clean2], axis=1);
data_fund_org_dummies_col_replace_grp

C:\Users\dmbog\AppData\Local\Temp\ipykernel_17512\1984298359.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_fund_org_dummies_col_replace_grp[elt] = data_fund_org_dummies_col_replace.loc[:, col_array].sum(axis=1).to_frame();
C:\Users\dmbog\AppData\Local\Temp\ipykernel_17512\1984298359.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_fund_org_dummies_col_replace_grp[elt] = data_fund_org_dummies_col_replace.loc[:, col_array].sum(axis=1).to_frame();
C:\Users\dmbog\AppData\Local\Temp\ipykernel_17512\1984298359.py:6:

,,german research foundation - germany,ku leuven internal funds - belgium,academy of finland - finland,administration for strategic preparedness and response - usa,"aethlon medical, inc. - usa",african academy of sciences - african union,african field epidemiology network - nigeria,agence nationale de la recherche - france,japan agency for medical research and development - japan,aix-marseille universite - france,...,university of california - usa,department of science and innovation - south africa,wellcome trust - united kingdom,werner siemens foundation - germany,whitehead institute for biomedical research - usa,william and ella owens medical research foundation - usa,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000849602200001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300006,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# save data
data_fund_org_dummies_col_replace_grp.to_csv(f'../TABLES/data_fund_org_clean_dummies_col_replace_grp.csv')

In [19]:
 # rename organisation names on columns
old_names = data_fund_org_clean['Funding Name Preferred'].to_list()
new_names  = data_fund_org_clean['Country'].to_list()
data_fund_country_dummies_col_replace = data_fund_org_dummies.rename(columns=dict(zip(old_names, new_names)))#.astype(int)    
data_fund_country_dummies_col_replace

,,germany,belgium,finland,usa,usa,african union,nigeria,nigeria,france,japan,...,multilateral,usa,usa,multilateral,multilateral,multilateral,multilateral,china,china,china
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000849602200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data_fund_country_dummies_col_replace.columns.value_counts()

usa                504
japan               95
germany             60
china               56
canada              32
united kingdom      32
european union      24
france              14
/                   10
australia            9
south africa         9
singapore            9
multilateral         9
belgium              8
uganda               6
russia               5
spain                5
italy                5
denmark              4
czech republic       4
finland              4
gabon                3
switzerland          3
netherlands          3
india                3
saudi arabia         3
norway               3
singapore            3
austria              3
korea                3
kenya                2
honduras             2
slovakia             2
sweden               2
japan                2
african union        2
nigeria              2
serbia               2
georgia              2
finland              2
jordan               2
guinea               2
denmark-germany      1
mexico     

In [21]:
# merge and sum the columns with the same countries name
data_fund_country_dummies_col_replace_grp = pd.DataFrame()
df_org_clean2 = pd.DataFrame()
for elt in  data_fund_country_dummies_col_replace.columns.unique():
    col_array = data_fund_country_dummies_col_replace.columns.str.fullmatch(elt) # contains
    data_fund_country_dummies_col_replace_grp[elt] = data_fund_country_dummies_col_replace.loc[:, col_array].sum(axis=1).to_frame();
data_fund_country_dummies_col_replace_grp = pd.concat([data_fund_country_dummies_col_replace_grp, df_org_clean2], axis=1);
data_fund_country_dummies_col_replace_grp

,,germany,belgium,finland,usa,african union,nigeria,france,japan,united kingdom,european union,...,turkey,serbia,russia,south africa,japan,sweden,slovakia,australia,colombia,georgia
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000849602200001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300006,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
data_fund_country_and_org_dummies_col_replace_grp = pd.concat([data_fund_country_dummies_col_replace_grp,data_fund_org_dummies_col_replace_grp], axis=1)

In [23]:
# save data
data_fund_country_and_org_dummies_col_replace_grp.to_csv(f'../TABLES/data_fund_country_and_org_clean_dummies_col_replace_grp.csv')

In [24]:
 # rename organisation names on columns
old_names = data_fund_org_clean['Funding Name Preferred'].to_list()
new_names  = data_fund_org_clean['region'].to_list()
data_fund_region_dummies_col_replace = data_fund_org_dummies.rename(columns=dict(zip(old_names, new_names)))#.astype(int)    
data_fund_region_dummies_col_replace

,,europe and central asia,europe and central asia,europe and central asia,north america,north america,africa,sub-saharan africa,sub-saharan africa,europe and central asia,east asia and pacific,...,multilateral,north america,north america,multilateral,multilateral,multilateral,multilateral,east asia and pacific,east asia and pacific,east asia and pacific
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000849602200001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data_fund_region_dummies_col_replace.columns.value_counts()

north america                      536
europe and central asia            186
east asia and pacific              178
sub-saharan africa                  27
multilateral                        12
/                                   10
middle east and north africa         5
latin america and the caribbean      5
south asia                           3
africa                               2
dtype: int64

In [26]:
# merge and sum the columns with the same countries name
data_fund_region_dummies_col_replace_grp = pd.DataFrame()
df_org_clean2 = pd.DataFrame()
for elt in  data_fund_region_dummies_col_replace.columns.unique():
    col_array = data_fund_region_dummies_col_replace.columns.str.fullmatch(elt) # contains
    data_fund_region_dummies_col_replace_grp[elt] = data_fund_region_dummies_col_replace.loc[:, col_array].sum(axis=1).to_frame();
data_fund_region_dummies_col_replace_grp = pd.concat([data_fund_region_dummies_col_replace_grp, df_org_clean2], axis=1);
data_fund_region_dummies_col_replace_grp

,,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,/,middle east and north africa,south asia,latin america and the caribbean,multilateral
UT (Unique WOS ID),Funding Name Preferred,,,,,,,,,,
WOS:000904661800001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000921279800001,National Institutes of Health (NIH)(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA); NIH(United States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA),0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000909001200001,Russian Science Foundation(Russian Science Foundation (RSF));,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:000777045900001,"National Research Foundation of Korea (NRF) - Ministry of Education(National Research Foundation of KoreaMinistry of Education (MOE), Republic of Korea); National Research Foundation of Korea (NRF) - Korean government (MSIT)(National Research Foundation of KoreaMinistry of Science & ICT (MSIT), Republic of Korea)",0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
WOS:000849602200001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300001,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WOS:A1968A855300006,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
data_fund_region_country_and_org_dummies_col_replace_grp = pd.concat([data_fund_region_dummies_col_replace_grp,
                                                                      data_fund_country_dummies_col_replace_grp,
                                                                      data_fund_org_dummies_col_replace_grp], axis=1)

In [28]:
# save data
data_fund_region_country_and_org_dummies_col_replace_grp.to_csv(f'../TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')

In [29]:
vvv

NameError: name 'vvv' is not defined

# The topics of Marburg Virus Diseases publications 

## proportion all

In [ ]:
countries_topics_list_all = data_and_topics_number.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_all.rename(columns = {0:'all_is_no_oa', 
#                               1:'all_is_oa'}, inplace = True)

countries_topics_list_all['all_total'] = countries_topics_list_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_all.columns = countries_topics_list_all.columns.astype(str)

for col in countries_topics_list_all.columns:
    countries_topics_list_all[col + '_%all'] = countries_topics_list_all[col]/countries_topics_list_all['all_total']*100
    
# save data
# countries_topics_list_all.to_csv(f'../TABLES/countries_all_oa_list.csv')
countries_topics_list_all

In [ ]:
# countries_topics_list_all.describe()

## proportion fund

In [ ]:
data_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 1]
data_fund.shape

In [ ]:
countries_topics_list_fund = data_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_fund.rename(columns = {0:'fund_is_no_oa', 
#                               1:'fund_is_oa'}, inplace = True)

countries_topics_list_fund['fund_total'] = countries_topics_list_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_fund.columns = countries_topics_list_fund.columns.astype(str)

for col in countries_topics_list_fund.columns:
    countries_topics_list_fund[col+'_%fund'] = countries_topics_list_fund[col]/countries_topics_list_fund['fund_total']*100

# save data
# countries_topics_list_fund.to_csv(f'../TABLES/countries_fund_oa_list.csv')

countries_topics_list_fund

## proportion no fund

In [ ]:
data_no_fund = data_and_topics_number.loc[data_and_topics_number['funding_yes'] == 0]
data_no_fund.head(2)

In [ ]:
countries_topics_list_no_fund = data_no_fund.reset_index().groupby(['Topic N°']).count().T
# countries_topics_list_no_fund.rename(columns = {0:'no_fund_is_no_oa', 
#                               1:'no_fund_is_oa'}, inplace = True)

countries_topics_list_no_fund['no_fund_total'] = countries_topics_list_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
countries_topics_list_no_fund.columns = countries_topics_list_no_fund.columns.astype(str)

for col in countries_topics_list_no_fund.columns:
    countries_topics_list_no_fund[col + '_%no_fund'] = countries_topics_list_no_fund[col]/countries_topics_list_no_fund['no_fund_total']*100
    
# countries_topics_list_no_fund['%_diff(%no_fund_is_oa-%no_fund_is_no_oa)'] = countries_topics_list_no_fund['%no_fund_is_oa'] - countries_topics_list_no_fund['%no_fund_is_no_oa']

# save data
# countries_topics_list_no_fund.to_csv(f'../TABLES/countries_fund_no_oa_list.csv')

countries_topics_list_no_fund

## concat all dataframe and save

In [ ]:
# concatenate dataframes on index
concat_countries_oa_list = pd.concat([countries_topics_list_all, 
                                        countries_topics_list_fund, 
                                        countries_topics_list_no_fund], 
                                       axis=1)
# save data
concat_countries_oa_list.to_csv(f'../TABLES/concat_countries_topics_list.csv')
concat_countries_oa_list

## significance

In [ ]:
for topics in range(10):
    fund_topics_percent = data_and_topics_number.loc[data_and_topics_number['Topic N°'] == topics]['funding_yes'].value_counts(normalize=True)[1]
    
    print(f'Funding percentage Topic {topics} : 'f'{fund_topics_percent}')

In [ ]:
# fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 1) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']
# no_fund_topics = data_and_topics_number.loc[(data_and_topics_number['funding_yes'] == 0) & (data_and_topics_number['Topic N°'] == 1)]['funding_yes']

In [ ]:
# test_sig(fund_topics, no_fund_topics)

# Topics Trends

In [ ]:
data_and_topics_number2 = data_and_topics_number[['Publication Year','funding_yes','Topic N°']].reset_index().set_index(['wos_ID', 'Publication Year','funding_yes'])
data_and_topics_number2

In [ ]:
data_topics_trend = pd.get_dummies(data_and_topics_number2['Topic N°'])
data_topics_trend

## ALL

In [ ]:
data_topics_trend_all = data_topics_trend.groupby('Publication Year').sum()

data_topics_trend_all['all_total'] = data_topics_trend_all[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_all.columns = data_topics_trend_all.columns.astype(str)

for col in data_topics_trend_all.columns:
    data_topics_trend_all[col + '_%all'] = data_topics_trend_all[col]/data_topics_trend_all['all_total']*100

data_topics_trend_all

## fund

In [ ]:
data_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==1].set_index(['wos_ID','funding_yes'])
data_fund.head(2)

In [ ]:
data_topics_trend_fund = data_fund.groupby('Publication Year').sum()

data_topics_trend_fund['fund_total'] = data_topics_trend_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_fund.columns = data_topics_trend_fund.columns.astype(str)

for col in data_topics_trend_fund.columns:
    data_topics_trend_fund[col + '_%fund'] = data_topics_trend_fund[col]/data_topics_trend_fund['fund_total']*100
    

data_topics_trend_fund

## no fund

In [ ]:
data_no_fund = data_topics_trend.reset_index().loc[data_topics_trend.reset_index()['funding_yes']==0].set_index(['wos_ID','funding_yes'])
data_no_fund.head(2)

In [ ]:
data_topics_trend_no_fund = data_no_fund.groupby('Publication Year').sum()

data_topics_trend_no_fund['no_fund_total'] = data_topics_trend_no_fund[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].sum(axis=1)
data_topics_trend_no_fund.columns = data_topics_trend_no_fund.columns.astype(str)

for col in data_topics_trend_no_fund.columns:
    data_topics_trend_no_fund[col + '_%no_fund'] = data_topics_trend_no_fund[col]/data_topics_trend_no_fund['no_fund_total']*100
    

data_topics_trend_no_fund

## concat and save

In [ ]:
# concatenate dataframes on index
concat_data_topics_trend = pd.concat([data_topics_trend_all, 
                                    data_topics_trend_fund, 
                                    data_topics_trend_no_fund], 
                                    axis=1)
# save data
concat_data_topics_trend.to_csv(f'../TABLES/concat_data_topics_trend.csv')
concat_data_topics_trend